# Run SIENA

To explain how to use SIENA we will use the __Islam__ dataset provided in ./data. The steps are all the same for any single cell dataset, however the parameterization in each step can very depending on what we want to do. 

In [2]:
from SIENA.siena import *
data = CSVDataset("./data/Islam", "Islam_treated.csv", labels_file="Islam_labels.csv")

### Differential expression analysis (4 steps):
### 1 - Create a model instance.
This is the step where we pass the counts, type and batch information (cell types, type of each cell, batch names and batch of each cell) and it is where we construct the model. To do so, we can specify if we want (or not) zero-inflation, gene-dispersion, batch-training (if so how many cells per mini-batch) and which prior to posit over the library scalings ($L_i$)

### 2 - Inference.
In this step, posterior approximations are generated for variables $\beta_{ig}$ and $L_i$, and $\theta_g$ are estimated. The optimization process takes $N$ number of iterations which can be specified.

As a first example, we'll create a model with gene-dispersion and log-normal library scalings, but with no zero-inflation and no batch-training during inference. 

In [11]:
N = 1000
model = ModelDE(data.X,data.cell_types,data.labels, zero_inflation=False, dispersion=True, use_log=True)
model.approximate_model(iters=N)

Using log scalings
Minisamples: 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 106s | Loss: 17642.791


To use a __Gamma__ prior over the library factors instead of a log-normal, we simply need to set false the use_log parameter.

In [5]:
N = 1000
model = ModelDE(data.X,data.cell_types,data.labels, zero_inflation=False, dispersion=True, use_log=False)
model.approximate_model(iters=N)

Minisamples: 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 107s | Loss: 17646.026


To use __mini-batches__ during inference, we only have to specify the number of cells per mini-batch, i.e., set minisample parameter. If minisample is None (default), SIENA uses the whole dataset in each update step.

In [7]:
N = 1000
model = ModelDE(data.X,data.cell_types,data.labels, zero_inflation=False, dispersion=True, use_log=True, minisample=46)
model.approximate_model(iters=N)

Using log scalings
Minisamples: 2
1000/1000 [100%] ██████████████████████████████ Elapsed: 127s | Loss: 17572.737


If the dataset contains __BATCH information__, we need to pass it in the model constructor. __Note that the batch information is only necessary for the DE tests__. To exemplify we'll <font color=blue>simulate a batch assignement</font> for the Islam dataset.

In [9]:
_batches = np.array(["batch 1", "batch 2"])
_batch_idx = np.zeros((92,))
_batch_idx[24:48] = 1
_batch_idx[70:92] = 1
print(_batch_idx)

N = 1000
model = ModelDE(data.X,data.cell_types,data.labels, batches=_batches, batch_idx=_batch_idx, 
                zero_inflation=False, dispersion=True, use_log=True)
model.approximate_model(iters=N)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Using log scalings
Minisamples: 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 107s | Loss: 17659.534


### 3 - Calculate the differential scores for each gene.
In this step, average Bayes factors are calculated for each gene, using all or a subset of valid cell pairs and a given number of Monte Carlo samples for each $\rho_{ig}$. If the model object was created with batch assignments, the DE scores are computed taking into account such information. See the preprint for more details.

In the following example we use __ALL pairs__ and 100 MC samples for each $\rho_{ig}$. 


In [17]:
scores = model.differential_expression_scores("MEF","ESC", n_samples=100)
print(scores)

rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
[ 1.48844605 -1.43643149  0.45411687 ...  0.04989049  0.08537334
 -0.66893374]


If we wish to use a __SUBSET of pairs__, then we need to specify the number of pairs to use.

In [15]:
scores = model.differential_expression_scores("MEF","ESC", n_samples=100, n_pairs=500)
print(scores)

rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
[ 1.56169842 -1.73404099  0.44343199 ...  0.03968778  0.08409054
 -0.69380853]


### 4 - Rank and select (optional) genes based on the DE scores.
To obtain the __RANKING with all genes__, we simply pass the gene identifiers and the scores. Note that the ids and scores must be in the same order. If the CSVDataset object is used then that is already fulfilled.

In [19]:
rank = model.differentially_expressed_genes(data.gene_names, scores)
print(rank.head(n=5))

        Gene    factor
6583    Bex1  9.922237
2547  Pou5f1  9.685674
3597   Thbs1  8.850172
6555    Bex4  8.517200
5814  Cyp2e1  8.439323


If we want to chose the __DEG__, a treshold must be specified (~ 2 or 3).

In [22]:
DEG = model.differentially_expressed_genes(data.gene_names, scores, threshold=2)
print(DEG.head(n=5))

        Gene    factor
6583    Bex1  9.922237
2547  Pou5f1  9.685674
3597   Thbs1  8.850172
6555    Bex4  8.517200
5814  Cyp2e1  8.439323


# Assess multiple AUC

To assess the __mean AUC__ of SIENA, we need to repeat the described pipeline multiple times and average the AUC obtained in each run. This is the same process as the one used in the paper to calculate SIENA's AUC means.
Here we exemplify the procedure for the __Islam__ dataset using only 2 runs.

First let us define a function to calculate the AUC of each run, using the sklearn package.

In [23]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def calculate_auc(scrs, lbls, trueDEG):
    scores = np.array(scrs)
    labels = np.array(lbls.isin(trueDEG))
    fpr,tpr,thresh = roc_curve(labels, scores)
    return auc(fpr, tpr)

Now we can perfrom and calculate the AUC of the 2 runs, as well as the mean AUC 

In [25]:
n_times = 2 #number of runs
n_epochs = 1000
path = './data/Islam'
trueDEG = pd.read_csv(path + '/trueDEG.txt', sep=",", header=0)
data = CSVDataset(path, "Islam_treated.csv", labels_file="Islam_labels.csv")
aucs = []
for i in range(n_times):
    model = ModelDE(data.X,data.cell_types,data.labels, zero_inflation=False, dispersion=True, use_log=True)
    model.approximate_model(iters=n_epochs)
    scores = model.differential_expression_scores("MEF","ESC", n_samples=100)
    res = model.differentially_expressed_genes(data.gene_names,scores)
    auc_myModel = calculate_auc(res['factor'], res["Gene"], trueDEG["SYMBOL"])
    aucs.append(auc_myModel)
print('AUCs: ' + str(aucs))
print('Average AUC: ' + str(np.mean(np.array(aucs))))

Using log scalings
Minisamples: 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 107s | Loss: 17626.478
rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
Using log scalings
Minisamples: 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 107s | Loss: 17621.376
rho1_values shape: (100, 44, 6757)
rho2_values shape: (100, 48, 6757)
AUCs: [0.6891779283676716, 0.6919780843643835]
Average AUC: 0.6905780063660275


For the __synthetic__ data the procedure is almost the same, with some small changes.
Here we exemplify the procedure with the 50-50-50-50 dataset.

In [4]:
n_times = 2 #number of runs
n_epochs = 1000
path = './data/Synthetic/50-50-50-50-400-400'
trueDEG = pd.read_csv(path + '/trueDEG.txt', sep=",", header=0)
data = CSVDataset(path, "synthetic.csv", gene_by_cell=True, labels_file="labels.csv")
aucs = []
for i in range(n_times):
    model = ModelDE(data.X,data.cell_types,data.labels, zero_inflation=False, dispersion=False, use_log=True)
    model.approximate_model(iters=n_epochs)
    scores = model.differential_expression_scores("B","A", n_samples=100)
    res = model.differentially_expressed_genes(data.gene_names,scores)
    auc_myModel = calculate_auc(res['factor'], res["Gene"], trueDEG["Gene"])
    aucs.append(auc_myModel)
print(aucs)
print('Average AUC: ' + str(np.mean(np.array(aucs))))

Using log scalings
No Dispersion
Minisamples: 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 82s | Loss: 24688.642
rho1_values shape: (100, 500, 1000)
rho2_values shape: (100, 500, 1000)
Using log scalings
No Dispersion
Minisamples: 1
1000/1000 [100%] ██████████████████████████████ Elapsed: 82s | Loss: 24241.558
rho1_values shape: (100, 500, 1000)
rho2_values shape: (100, 500, 1000)
[0.92149375, 0.9137875]
Average AUC: 0.917640625
